## correct things in theme graph

Correct things found in the theme/genre graph


In [1]:
# import packages

from collections import defaultdict, OrderedDict, Counter
from datetime import datetime

from dateutil.relativedelta import relativedelta
import glob
from itertools import product, combinations
import matplotlib.pyplot as plt
import numpy    as np
#import networkx as nx
from operator import itemgetter
import pandas   as pd
import rdflib as rdflib
import re
from scipy.signal import convolve2d
import seaborn as sns
from sklearn.preprocessing import MultiLabelBinarizer, minmax_scale
from SPARQLWrapper import SPARQLWrapper, JSON, POST, TURTLE
import sys
from rdflib.namespace import XSD, Namespace
from rdflib.term import URIRef

import IPython
import pprint
pp = pprint.PrettyPrinter(indent=4)



from constants import *
from helpers import *

In [2]:
# 
from rdflib import Graph

from rdflib.namespace import RDF, SKOS
rel = rdflib.Namespace('http://ldf.fi/relse/')


In [3]:
# colors
import matplotlib.colors as mcolors
from matplotlib import cm
# set style
import seaborn as sns

# set theme for visualizations
sns.set_theme(style="white",palette="PuOr_r")

In [44]:
# cluster per genre information
# load genre graphs
l = Graph() 
l.parse("../../data/finnish_genre_theme.nt",format="nt")

k = Graph() 
k.parse("../../data/translated_genre_theme.nt",format="nt")

<Graph identifier=N5dfcfa9489f14d8fa8cf9daab2c991a1 (<class 'rdflib.graph.Graph'>)>

In [45]:
# initial language lookup

qres = k.query("""
#PREFIX kaunokki: <http://www.yso.fi/onto/kaunokki#> 
PREFIX rel: <http://ldf.fi/relse/> 
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX xml: <http://www.w3.org/XML/1998/namespace>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT ?lang
WHERE {
  ?teos a rel:novel ; rel:langOrig ?lang. 
  
}  """)

langlabel_lookup = {}

for l in qres:
    lang=l["lang"].toPython()
    #print(lang)
    if lang not in langlabel_lookup.keys():
        langlabel_lookup[lang]= lang.split("/")[-1]
    else:
        print("oop",lang)

In [46]:
# print out tha language labels
print(langlabel_lookup.items())

dict_items([('http://lexvo.org/id/iso639-3/eng', 'eng'), ('http://lexvo.org/id/iso639-3/ita', 'ita'), ('http://lexvo.org/id/iso639-3/deu', 'deu'), ('http://lexvo.org/id/iso639-3/isl', 'isl'), ('http://lexvo.org/id/iso639-3/swe', 'swe'), ('http://lexvo.org/id/iso639-3/rus', 'rus'), ('http://lexvo.org/id/iso639-3/fra', 'fra'), ('http://lexvo.org/id/iso639-3/dan', 'dan'), ('http://lexvo.org/id/iso639-3/spa', 'spa'), ('http://lexvo.org/id/iso639-3/tur', 'tur'), ('http://lexvo.org/id/iso639-3/ces', 'ces'), ('http://lexvo.org/id/iso639-3/por', 'por'), ('http://lexvo.org/id/iso639-3/nor', 'nor'), ('http://lexvo.org/id/iso639-3/hin', 'hin'), ('http://lexvo.org/id/iso639-3/kor', 'kor'), ('http://lexvo.org/id/iso639-3/lav', 'lav'), ('http://lexvo.org/id/iso639-3/est', 'est'), ('http://lexvo.org/id/iso639-3/yid', 'yid'), ('http://lexvo.org/id/iso639-3/hun', 'hun'), ('http://lexvo.org/id/iso639-3/nld', 'nld'), ('http://lexvo.org/id/iso639-3/pol', 'pol'), ('http://lexvo.org/id/iso639-3/kat', 'kat')

In [49]:
oldlang=[]
for key,v in langlabel_lookup.items():
    if "lexvo" not in key:
        print(key,v)
        oldlang.append(key)

http://www.lingvoj.org/lang/be-x-old be-x-old
http://www.lingvoj.org/lang/bnt bnt
http://www.lingvoj.org/lang/smi smi
http://www.lingvoj.org/lang/en-gb en-gb
http://www.lingvoj.org/lang/fiu fiu


In [50]:
for key,v in langlabel_lookup.items():
    if "vec" in key:
        print(key,v)

http://lexvo.org/id/iso639-3/vec vec


In [51]:
# correct list
# smi http://www.lexvo.org/page/iso639-5/smi
# en-gb http://lexvo.org/id/iso639-3/eng
# bnt http://www.lexvo.org/page/iso639-5/bnt
# fiu http://www.lexvo.org/page/iso639-5/fiu
# be-x-old http://www.lexvo.org/page/iso639-3/bel
# 'http://lexvo.org/id/iso639-3/vec': 'vec',

#newlang=["http://www.lexvo.org/page/iso639-5/smi","http://lexvo.org/id/iso639-3/eng","http://www.lexvo.org/page/iso639-5/bnt",
#        "http://www.lexvo.org/page/iso639-5/fiu","http://www.lexvo.org/page/iso639-3/bel"]
#updated= ["http://lexvo.org/page/iso639-5/smi","http://lexvo.org/id/iso639-3/eng","http://lexvo.org/page/iso639-5/bnt",
#        "http://lexvo.org/page/iso639-5/fiu","http://lexvo.org/page/iso639-3/bel"]
updated= ["http://lexvo.org/page/iso639-5/smi","http://lexvo.org/id/iso639-3/eng","http://lexvo.org/page/iso639-5/bnt",
        "http://lexvo.org/page/iso639-5/fiu"]
newlang={"http://www.lingvoj.org/lang/be-x-old":"http://lexvo.org/page/iso639-3/bel","http://www.lingvoj.org/lang/bnt":"http://lexvo.org/page/iso639-5/bnt","http://www.lingvoj.org/lang/smi":"http://lexvo.org/id/iso639-5/smi","http://www.lingvoj.org/lang/fiu":"http://www.lexvo.org/page/iso639-5/fiu","http://www.lingvoj.org/lang/en-gb":"http://lexvo.org/id/iso639-3/eng","http://lexvo.org/id/iso639-3/vec":'http://lexvo.org/id/iso639-3/rus'}

In [52]:
for key,val in newlang.items():
    
    print(key,val)
    old= key
    new=val.replace("page","id")
    print(old,new)
    query="""
    PREFIX rel: <http://ldf.fi/relse/>
             DELETE {?teos rel:langOrig <OLD> }
             INSERT { ?teos rel:langOrig  <NEW> }
             WHERE { ?teos rel:langOrig <OLD> }
             """
    q=query.replace("OLD",old).replace("NEW",new)
    k.update(q)
    #print("After second update:")
    

http://www.lingvoj.org/lang/be-x-old http://lexvo.org/page/iso639-3/bel
http://www.lingvoj.org/lang/be-x-old http://lexvo.org/id/iso639-3/bel
http://www.lingvoj.org/lang/bnt http://lexvo.org/page/iso639-5/bnt
http://www.lingvoj.org/lang/bnt http://lexvo.org/id/iso639-5/bnt
http://www.lingvoj.org/lang/smi http://lexvo.org/id/iso639-5/smi
http://www.lingvoj.org/lang/smi http://lexvo.org/id/iso639-5/smi
http://www.lingvoj.org/lang/fiu http://www.lexvo.org/page/iso639-5/fiu
http://www.lingvoj.org/lang/fiu http://www.lexvo.org/id/iso639-5/fiu
http://www.lingvoj.org/lang/en-gb http://lexvo.org/id/iso639-3/eng
http://www.lingvoj.org/lang/en-gb http://lexvo.org/id/iso639-3/eng
http://lexvo.org/id/iso639-3/vec http://lexvo.org/id/iso639-3/rus
http://lexvo.org/id/iso639-3/vec http://lexvo.org/id/iso639-3/rus


In [53]:
# save
k.serialize(destination="../../data/foreign_theme_genre.nt",format="nt")


/u/98/peurat1/unix/kirjasampo/benv/lib/python3.8/site-packages/rdflib/plugins/serializers/nt.py:36: UserWarning: NTSerializer always uses UTF-8 encoding. Given encoding was: None
  warnings.warn(


<Graph identifier=N5dfcfa9489f14d8fa8cf9daab2c991a1 (<class 'rdflib.graph.Graph'>)>